# Hosting Strands Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime / 在 Amazon Bedrock AgentCore Runtime 中托管使用 Amazon Bedrock 模型的 Strands Agents

## Overview / 概述

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. We will provide examples using Amazon Bedrock models and non-Bedrock models such as Azure OpenAI and Gemini.

在本教程中，我们将学习如何使用 Amazon Bedrock AgentCore Runtime 托管您现有的代理。我们将提供使用 Amazon Bedrock 模型和非 Bedrock 模型（如 Azure OpenAI 和 Gemini）的示例。


### Tutorial Details / 教程详情


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.7                                                        |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |

### Tutorial Architecture / 教程架构

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a Strands Agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

在本教程中，我们将描述如何将现有代理部署到 AgentCore 运行时。

出于演示目的，我们将使用一个使用 Amazon Bedrock 模型的 Strands Agent

在我们的示例中，我们将使用一个非常简单的代理，它有两个工具：`get_weather` 和 `get_time`。

<img src="./images/architecture_runtime.png" width="50%"/>


### Tutorial Key Features / 教程关键特性

* 在 Amazon Bedrock AgentCore Runtime 上托管代理
* 使用 Amazon Bedrock 模型
* 使用 Strands Agents



## Creating your agents and experimenting locally / 创建代理并在本地进行实验

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

在将代理部署到 AgentCore Runtime 之前，让我们在本地开发和运行它们以进行实验。

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

对于生产级代理应用程序，我们需要将代理创建过程与代理调用过程解耦。使用 AgentCore Runtime，我们将用 `@app.entrypoint` 装饰器装饰代理的调用部分，并将其作为运行时的入口点。让我们首先看看在实验阶段如何开发每个代理。

The architecture here will look as following:

这里的架构如下所示：

<img src="./images/architecture_local.png" width="50%"/>


In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models import BedrockModel

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.amazon.nova-pro-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))

Overwriting strands_claude.py


#### Invoking local agent / 调用本地代理

In [15]:
!python3 strands_claude.py '{"prompt": "What is the weather now?"}'

I can check the current weather for you. Let me do that right away.
Tool #1: weather
It's currently sunny outside! Perfect weather to enjoy some time outdoors if you're able to.

## Preparing your agent for deployment on AgentCore Runtime / 准备在 AgentCore Runtime 上部署代理

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

现在让我们将代理部署到 AgentCore Runtime。为此我们需要：
* 使用 `from bedrock_agentcore.runtime import BedrockAgentCoreApp` 导入 Runtime App
* 在代码中使用 `app = BedrockAgentCoreApp()` 初始化 App
* 使用 `@app.entrypoint` 装饰器装饰调用函数
* 让 AgentCoreRuntime 使用 `app.run()` 控制代理的运行

### Strands Agents with Amazon Bedrock model / 使用 Amazon Bedrock 模型的 Strands Agents
Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

让我们从使用 Amazon Bedrock 模型的 Strands Agent 开始。所有其他的都将以完全相同的方式工作。

In [16]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Overwriting strands_claude.py


## What happens behind the scenes? / 幕后发生了什么？

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

当您使用 `BedrockAgentCoreApp` 时，它会自动：

* 创建一个监听端口 8080 的 HTTP 服务器
* 实现处理代理需求所需的 `/invocations` 端点
* 实现用于健康检查的 `/ping` 端点（对异步代理非常重要）
* 处理适当的内容类型和响应格式
* 根据 AWS 标准管理错误处理

## Deploying the agent to AgentCore Runtime / 将代理部署到 AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

`CreateAgentRuntime` 操作支持全面的配置选项，让您可以指定容器镜像、环境变量和加密设置。您还可以配置协议设置（HTTP、MCP）和授权机制来控制客户端与代理的通信方式。

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

**注意：** 运维最佳实践是将代码打包为容器并使用 CI/CD 管道和 IaC 推送到 ECR

In this tutorial can will the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

在本教程中，我们将使用 Amazon Bedrock AgentCore Python SDK 轻松打包您的工件并将其部署到 AgentCore 运行时。

### Configure AgentCore Runtime deployment / 配置 AgentCore Runtime 部署

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

首先，我们将使用启动工具包来配置 AgentCore Runtime 部署，包括入口点、我们刚创建的执行角色和需求文件。我们还将配置启动工具包在启动时自动创建 Amazon ECR 存储库。

During the configure step, your docker file will be generated based on your application code

在配置步骤中，将根据您的应用程序代码生成 docker 文件

 <img src="images/configure.png" width="60%"/>


In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import os

region = os.getenv('AWS_REGION')
boto_session = Session(region_name=region)

agentcore_runtime = Runtime()
agent_name = "strands_claude_getting_started_v2"
response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/workshop/ImmersionDay-SYE/Agentcore-Experiment/strands_claude.py, bedrock_agentcore_name=strands_claude
Configuring BedrockAgentCore agent: strands_claude_getting_started_v2


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /workshop/ImmersionDay-SYE/Agentcore-Experiment/Dockerfile
Generated .dockerignore: /workshop/ImmersionDay-SYE/Agentcore-Experiment/.dockerignore
Changing default agent from 'strands_claude_getting_started' to 'strands_claude_getting_started_v2'
Bedrock AgentCore configured: /workshop/ImmersionDay-SYE/Agentcore-Experiment/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/workshop/ImmersionDay-SYE/Agentcore-Experiment/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/workshop/ImmersionDay-SYE/Agentcore-Experiment/Dockerfile'), dockerignore_path=PosixPath('/workshop/ImmersionDay-SYE/Agentcore-Experiment/.dockerignore'), runtime='None', region='us-east-1', account_id='983760593521', execution_role=None, ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime / 将代理启动到 AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

现在我们有了 docker 文件，让我们将代理启动到 AgentCore Runtime。这将创建 Amazon ECR 存储库和 AgentCore Runtime

<img src="images/launch.png" width="75%"/>


In [ ]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_getting_started_v2' to account 983760593521 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_getting_started_v2
✅ ECR repository available: 983760593521.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_getting_started_v2
Getting or creating execution role for agent: strands_claude_getting_started_v2
Using AWS region: us-east-1, account ID: 983760593521
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-103201c6cb


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-strands_claude_getting_started_v2


Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-103201c6cb
Starting execution role creation process for agent: strands_claude_getting_started_v2
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-east-1-103201c6cb
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-103201c6cb
✓ Role created: arn:aws:iam::983760593521:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-103201c6cb
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-strands_claude_getting_started_v2
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::983760593521:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-103201c6cb
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_claude_getting_started_v2
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-103201c6cb
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCode

### Checking for the AgentCore Runtime Status / 检查 AgentCore Runtime 状态
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

现在我们已经部署了 AgentCore Runtime，让我们检查其部署状态

In [21]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude_getting_started_v2


'READY'

### Invoking AgentCore Runtime / 调用 AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

最后，我们可以使用有效负载调用我们的 AgentCore Runtime

<img src="./images/invoke.png" width="75%"/>


In [22]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

Invoking BedrockAgentCore agent 'strands_claude_getting_started_v2' via cloud endpoint


{'ResponseMetadata': {'RequestId': 'a9f9d901-d6a5-48ce-80de-0dcb628fb7d7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 04 Sep 2025 00:34:59 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a9f9d901-d6a5-48ce-80de-0dcb628fb7d7',
   'baggage': 'Self=1-68b8dea7-74ac324949d2faa903cc18e0,session.id=cbcc5c75-062f-4cf7-a192-9886a6036cd3',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'cbcc5c75-062f-4cf7-a192-9886a6036cd3',
   'x-amzn-trace-id': 'Root=1-68b8dea7-0cf0a44241253c54291857bc;Self=1-68b8dea7-74ac324949d2faa903cc18e0'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'cbcc5c75-062f-4cf7-a192-9886a6036cd3',
 'traceId': 'Root=1-68b8dea7-0cf0a44241253c54291857bc;Self=1-68b8dea7-74ac324949d2faa903cc18e0',
 'baggage': 'Self=1-68b8dea7-74ac324949d2faa903cc18e0,session.id=cbcc5c75-062f-4cf7-a192-9886a6036cd3',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'"The weather

### Processing invocation results / 处理调用结果

We can now process our invocation results to include it in an application

现在我们可以处理调用结果以将其包含在应用程序中

In [26]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(f'**{response_text}**'))

**The weather is currently sunny! It's a beautiful day outside.**

### Invoking AgentCore Runtime with boto3 / 使用 boto3 调用 AgentCore Runtime

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

现在您的 AgentCore Runtime 已创建，您可以使用任何 AWS SDK 调用它。例如，您可以使用 boto3 的 `invoke_agent_runtime` 方法。

In [27]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

The answer to 2+2 is 4.

## Cleanup (Optional) / 清理（可选）

Let's now clean up the AgentCore Runtime created

现在让我们清理创建的 AgentCore Runtime

In [28]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

('983760593521.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_getting_started_v2',
 'strands_claude_getting_started_v2-j3S16P3fYV',
 'bedrock-agentcore-strands_claude_getting_started_v2')

In [29]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations! / 恭喜！